<a href="https://colab.research.google.com/github/claudio1975/Medium-blog/blob/master/Scikit-LLM/Flood_Prediction/Data_Augmentation/Flood_Dataset_Features_Generation_work_v_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar  8 07:46:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
# install scikit-llm library
!pip install scikit-llm==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.13.3
    Uninstalling openai-1.13.3:
      Successfully uninstalled openai-1.13.3


In [ ]:

# upload the used libraries
from skllm.config import SKLLMConfig
import numpy as np
import pandas as pd
from skllm import ZeroShotGPTClassifier
from skllm.preprocessing import GPTVectorizer
import os
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# upload OpenAI API keys
os.environ["OPENAI_SECRET_KEY"] ="OPENAI_SECRET_KEY"
OPENAI_SECRET_KEY=os.environ["OPENAI_SECRET_KEY"]
SKLLMConfig.set_openai_key(OPENAI_SECRET_KEY)


In [ ]:
# upload data set
df=pd.read_csv("./flood_dataset.csv",index_col=0)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38398 entries, 0 to 38397
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EVENT_ID           38398 non-null  int64  
 1   BEGIN_DATE_TIME    38398 non-null  object 
 2   END_DATE_TIME      38398 non-null  object 
 3   STATE              38398 non-null  object 
 4   STATE_FIPS         38398 non-null  float64
 5   EVENT_TYPE         38398 non-null  object 
 6   CZ_TYPE            38398 non-null  object 
 7   CZ_FIPS            38398 non-null  int64  
 8   CZ_NAME            38398 non-null  object 
 9   WFO                38398 non-null  object 
 10  CZ_TIMEZONE        38398 non-null  object 
 11  INJURIES_DIRECT    38398 non-null  int64  
 12  INJURIES_INDIRECT  38398 non-null  int64  
 13  DEATHS_DIRECT      38398 non-null  int64  
 14  DEATHS_INDIRECT    38398 non-null  int64  
 15  DAMAGE_PROPERTY    38398 non-null  float64
 16  DAMAGE_CROPS       383

### Zero Shot Text Classification

In [ ]:
# select text column to fit zero-shot classification (sentiment and risk on episode_narrative)
X_sentiment=df.iloc[:,29]

In [ ]:
# fit the model
clf1 = ZeroShotGPTClassifier(openai_model="gpt-3.5-turbo-0125")
clf1.fit(None, ["positive", "negative", "neutral"])
predicted_sentiment_gpt = pd.DataFrame(clf1.predict(X_sentiment), columns=['predicted_sentiment_gpt'])

100%|██████████| 38398/38398 [5:23:34<00:00,  1.98it/s]


In [ ]:
# fit the model
clf2 = ZeroShotGPTClassifier(openai_model="gpt-3.5-turbo-0125")
clf2.fit(None, ["low risk", "medium risk", "high risk"])
predicted_risk_gpt = pd.DataFrame(clf2.predict(X_sentiment), columns=['predicted_risk_gpt'])

100%|██████████| 38398/38398 [6:58:34<00:00,  1.53it/s]


### Embedding

In [ ]:
# select text column to build embedding
X_sentiment=df.iloc[:,29]
X_event=df.iloc[:,30]


In [ ]:
# build embedding on episode_narrative
model_episod = GPTVectorizer()
vectors_episod = model_episod.fit_transform(X_sentiment)[:,0:10]

100%|██████████| 38398/38398 [1:46:20<00:00,  6.02it/s]


In [ ]:
vectors_episod_gpt = pd.DataFrame(vectors_episod, columns=['embed_episod_gpt'+str(i) for i in range(vectors_episod.shape[1])])

In [ ]:
vectors_episod_gpt.head()

,embed_episod_gpt0,embed_episod_gpt1,embed_episod_gpt2,embed_episod_gpt3,embed_episod_gpt4,embed_episod_gpt5,embed_episod_gpt6,embed_episod_gpt7,embed_episod_gpt8,embed_episod_gpt9
0,-0.030351,-0.011216,-0.000069,-0.013918,-0.010865,0.031305,-0.005294,0.021519,0.000904,-0.035066
1,-0.030499,-0.013081,0.014321,-0.007386,0.006444,0.023179,-0.004329,-0.017636,-0.018074,-0.020209
2,-0.030499,-0.013081,0.014321,-0.007386,0.006444,0.023179,-0.004329,-0.017636,-0.018074,-0.020209
3,-0.030499,-0.013081,0.014321,-0.007386,0.006444,0.023179,-0.004329,-0.017636,-0.018074,-0.020209
4,-0.030500,-0.013082,0.014322,-0.007393,0.006412,0.023167,-0.004356,-0.017624,-0.018088,-0.020236


In [ ]:
# build embedding on event_narrative
model_event = GPTVectorizer()
vectors_event = model_event.fit_transform(X_event)[:,0:10]

100%|██████████| 38398/38398 [2:18:11<00:00,  4.63it/s]


In [ ]:
vectors_event_gpt = pd.DataFrame(vectors_event, columns=['embed_event_gpt'+str(i) for i in range(vectors_event.shape[1])])

In [ ]:
vectors_event_gpt.head()

,embed_event_gpt0,embed_event_gpt1,embed_event_gpt2,embed_event_gpt3,embed_event_gpt4,embed_event_gpt5,embed_event_gpt6,embed_event_gpt7,embed_event_gpt8,embed_event_gpt9
0,-0.005580,-0.009172,-0.000350,-0.001341,-0.020569,-0.002682,-0.017443,0.013211,-0.015320,-0.032622
1,-0.010368,-0.004088,-0.001276,0.008959,-0.041108,0.024286,-0.024926,-0.009651,-0.005599,-0.019380
2,-0.008556,-0.010422,0.000788,0.008634,-0.009571,0.031676,-0.023744,-0.016657,-0.027255,-0.015474
3,-0.019469,-0.015229,0.002395,0.007543,0.007504,0.027961,-0.023019,-0.020938,-0.020496,-0.022239
4,-0.019469,-0.015229,0.002395,0.007543,0.007504,0.027961,-0.023019,-0.020938,-0.020496,-0.022239


In [ ]:
# build a new data set with predictions
df_fe= pd.concat([df,predicted_sentiment_gpt.set_index(df.index),predicted_risk_gpt.set_index(df.index),vectors_episod_gpt.set_index(df.index),vectors_event_gpt.set_index(df.index)], axis=1)

In [ ]:
df_fe.head()

,EVENT_ID,BEGIN_DATE_TIME,END_DATE_TIME,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,...,embed_event_gpt1,embed_event_gpt2,embed_event_gpt3,embed_event_gpt4,embed_event_gpt5,embed_event_gpt6,embed_event_gpt7,embed_event_gpt8,embed_event_gpt9,predicted_risk_gpt
0,2161,28-OCT-06 14:00:00,28-OCT-06 16:30:00,MASSACHUSETTS,25.0,Flood,C,25,SUFFOLK,BOX,...,-0.009172,-0.000350,-0.001341,-0.020569,-0.002682,-0.017443,0.013211,-0.015320,-0.032622,high risk
1,4734,16-NOV-06 20:00:00,16-NOV-06 22:00:00,NEW YORK,36.0,Flood,C,43,HERKIMER,ALY,...,-0.004088,-0.001276,0.008959,-0.041108,0.024286,-0.024926,-0.009651,-0.005599,-0.019380,high risk
2,3709,16-NOV-06 23:49:00,17-NOV-06 05:28:00,NEW YORK,36.0,Flood,C,57,MONTGOMERY,ALY,...,-0.010422,0.000788,0.008634,-0.009571,0.031676,-0.023744,-0.016657,-0.027255,-0.015474,high risk
3,3718,17-NOV-06 05:03:00,17-NOV-06 13:56:00,NEW YORK,36.0,Flood,C,57,MONTGOMERY,ALY,...,-0.015229,0.002395,0.007543,0.007504,0.027961,-0.023019,-0.020938,-0.020496,-0.022239,high risk
4,3720,17-NOV-06 05:03:00,17-NOV-06 13:56:00,NEW YORK,36.0,Flood,C,93,SCHENECTADY,ALY,...,-0.015229,0.002395,0.007543,0.007504,0.027961,-0.023019,-0.020938,-0.020496,-0.022239,high risk


In [ ]:
df_fe.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EVENT_ID,38398.0,NaN,NaN,NaN,582042.643888,293802.225946,350.0,320257.25,641838.5,814379.5,1095508.0
BEGIN_DATE_TIME,38398,27478,01-JUN-19 00:00:00,204,NaN,NaN,NaN,NaN,NaN,NaN,NaN
END_DATE_TIME,38398,25667,31-MAY-19 23:59:00,191,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STATE,38398,54,VIRGINIA,2581,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STATE_FIPS,38398.0,NaN,NaN,NaN,30.809313,17.77272,1.0,19.0,29.0,42.0,99.0
EVENT_TYPE,38398,1,Flood,38398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CZ_TYPE,38398,1,C,38398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CZ_FIPS,38398.0,NaN,NaN,NaN,88.147924,82.932916,1.0,29.0,71.0,127.0,840.0
CZ_NAME,38398,1754,WASHINGTON,469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WFO,38398,121,LWX,3289,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_fe.shape

(38398, 53)

In [ ]:
from google.colab import files
df_fe.to_csv('flood_dataset_fe4.csv')
files.download('flood_dataset_fe4.csv')